In [1]:
# importacoes
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import openpyxl
import time
import pandas as pd

In [3]:
# definindo lista de empresas:
lista_br = pd.read_excel('lista_hospitais_na_bolsa.xlsx')
tickers = lista_br['id_acao']
tickers

0    AALR3
1    DASA3
2    FLRY3
3    HAPV3
4    KRSA3
5    MATD3
6    ODPV3
7    ONCO3
8    QUAL3
9    RDOR3
Name: id_acao, dtype: object

In [4]:
# - 1 utilização do webdriver

browser = webdriver.Chrome()
time.sleep(2)
browser.get('https://statusinvest.com.br/acoes/')
time.sleep(15)

In [5]:
# remover propagandas
ad_close = browser.find_element(By.XPATH, '//button[@class="btn-close"]')
ad_close.click()
time.sleep(1)

In [7]:
# raspar os históricos de BALANCO
with pd.ExcelWriter('raspagem_tabela_balanco.xlsx', engine='openpyxl') as writer:
    for ticker in tickers:
        #Acessando conteúdo de cada empresa
        lupa = browser.find_element(By.XPATH, '//a[@class="main-search" and @title="Busca por nome de empresas, FIIs, índices e muito mais"]')
        lupa.click()
        input =  browser.find_element(By.XPATH, '//input[@class="Typeahead-input input tt-input"]')
        input.send_keys(ticker)
        time.sleep(3)
        input.send_keys(Keys.ARROW_DOWN)
        input.send_keys(Keys.ENTER)
        time.sleep(3)
        
        # SCROLLS para carregar página
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # clicar em TODOS ver mais para html carregar
        #detecta quantos ver mais tem:
        ver_mais = browser.find_elements(By.XPATH, '//button[@title="Ver tudo"]')
        len(ver_mais) # tem 3 ou 4 'ver mais'
        time.sleep(1)

        # gera loop para clicar em todos botoes 'ver mais':
        for botao in range(len(ver_mais)):
            # Rolar até o ver mais
            browser.execute_script("arguments[0].scrollIntoView(true);", ver_mais[botao])
            browser.execute_script("window.scrollBy(0, -400);")
            time.sleep(1)

            #clicar no ver mais
            ver_mais[botao].click()
            time.sleep(2)

        # simplificar tabelas (eliminar análises horizontais)
        # encontrar input que quero clicar
        # tem um botao verde para cada uma das tabelas
        botao_verde = browser.find_elements(By.XPATH, '//div[@data-tooltip="Apresentar/esconder colunas do grid"]')

        for botao in range(len(botao_verde)):
            browser.execute_script("arguments[0].scrollIntoView(true);", botao_verde[botao])
            time.sleep(1)
            browser.execute_script("window.scrollBy(0, -300);")
            time.sleep(1)
            input_bv1 = botao_verde[botao].find_element(By.TAG_NAME, 'input')
            input_bv1.click()
            #encontrar checkbox e clicar
            ul_bv = botao_verde[botao].find_element(By.TAG_NAME, 'ul')
            li = ul_bv.find_elements(By.TAG_NAME, 'li')
            for j in range(len(li)):
                if (li[j].text == 'AH'):
                    checkbox = li[j]
                    checkbox.click()
                    time.sleep(2)
            checkbox.send_keys(Keys.ESCAPE)
            time.sleep(1)
                    
        # divs resultado, fluxo, e balanço
        divs_resultado_e_fluxo = browser.find_elements(By.XPATH, '//div[@class="DRE table-info card card-panel white mb-5"]')

        div_resultado = divs_resultado_e_fluxo[0] # DIV RESULTADO É A PRIMEIRA
        div_resultado_inteira_text = div_resultado.text.split('\n')
        print(len(div_resultado_inteira_text)) # guardar - variável com textos da div com resultados

        div_fluxo = divs_resultado_e_fluxo[1] # DIV FLUXO É A SEGUNDA
        div_fluxo_inteira_text = div_fluxo.text.split('\n')
        print(len(div_fluxo_inteira_text)) # guardar - variável com textos da div com fluxo
        
        
        div_balanco = browser.find_element(By.XPATH, '//div[@aria-label="Grid com o Balanço Patrimonial"]')
        balanco_texto = div_balanco.text.split('\n')
        balanco_texto
        time.sleep(2)
        
        # Numero de anos
        divs_th = browser.find_elements(By.XPATH, '//th[@class="text-right DATA timeType-0"]') # o total de divs th com essa classe estão divididos entre as três tabelas com valores financeiros (resultado, fluxo de caixa e balanço patrimial)
        numero_de_anos = len(divs_th)/3
        numero_de_anos = int(numero_de_anos)
        anos = []
        for ano in range(len(divs_th)):
            #print(divs_th[ano].text)
            anos.append(divs_th[ano].text)
        print(anos)
            
        # Criar um conjunto para armazenar anos únicos
        anos_passados = set()

        # Lista para armazenar anos não repetidos
        anos_nao_repetidos = []

        # Percorrer a lista de anos
        for ano in anos:
        # Verificar se o ano já foi visitado
            if ano not in anos_passados:
                anos_nao_repetidos.append(ano)
                anos_passados.add(ano)
            else:
                # Se encontrar um ano repetido, parar o loop
                break

        # Imprimir a lista de anos não repetidos
        print(anos_nao_repetidos) # printa a array com os anos que irão compor o header do dataframe
        numero_de_anos = len(anos_nao_repetidos)
        # print(anos) 
        
        ### pegando valores conforme padrão detectado no listão de textos (BALANÇO):
        
        ativo_total = []
        indice_inicial = 3
        for texto in range(3,3 + numero_de_anos):
            ativo_total.append(balanco_texto[texto])
        print(ativo_total)

        ativo_circulante = []
        for texto in range(3 + numero_de_anos + 3, 3 + numero_de_anos + 3 + numero_de_anos):
            ativo_circulante.append(balanco_texto[texto])
        print(ativo_circulante)
        
        #time.sleep(2)
        
        aplicacoes_financeiras = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos):
            aplicacoes_financeiras.append(balanco_texto[texto])
        print(aplicacoes_financeiras)
        
       # time.sleep(2)
        
        caixa_e_equivalentes_de_caixa = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            caixa_e_equivalentes_de_caixa.append(balanco_texto[texto])
        print(caixa_e_equivalentes_de_caixa)
        
        #time.sleep(2)
        
        contas_a_receber = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            contas_a_receber.append(balanco_texto[texto])
        print(contas_a_receber)
            
        #time.sleep(2)
        
        estoque = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            estoque.append(balanco_texto[texto])
        print(estoque)

        ativo_nao_circulante = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos):
            ativo_nao_circulante.append(balanco_texto[texto])
        print(ativo_nao_circulante)
        
        #time.sleep(2)
        
        ativo_realizavel_a_longo_prazo = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos):
            ativo_realizavel_a_longo_prazo.append(balanco_texto[texto])
        print(ativo_realizavel_a_longo_prazo)
        
       #time.sleep(2)
        
        investimentos = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            investimentos.append(balanco_texto[texto])
        print(investimentos)
          
        #time.sleep(2)  
        
        imobilizado = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos):
            imobilizado.append(balanco_texto[texto])
        print(imobilizado)
        
        #time.sleep(2)
        
        intangivel = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos):
            intangivel.append(balanco_texto[texto])
        print(intangivel)
         
        #time.sleep(2)   
        
        passivo_total = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            passivo_total.append(balanco_texto[texto])
        print(passivo_total)
        
        #time.sleep(2) 
           
        passivo_circulante = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            passivo_circulante.append(balanco_texto[texto])
        print(passivo_circulante)

        passivo_nao_circulante = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            passivo_nao_circulante.append(balanco_texto[texto])
        print(passivo_nao_circulante)

        patrimonio_liquido_consolidado = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            patrimonio_liquido_consolidado.append(balanco_texto[texto])
        print(patrimonio_liquido_consolidado)

        capital_social_realizado = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            capital_social_realizado.append(balanco_texto[texto])
        print(capital_social_realizado)

        reserva_capital = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            reserva_capital.append(balanco_texto[texto])
        print(reserva_capital)

        reserva_lucros = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            reserva_lucros.append(balanco_texto[texto])
        print(reserva_lucros)

        participacao_dos_nao_controladores = []
        for texto in range(3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2, 3 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 3 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos + 2 + numero_de_anos):
            participacao_dos_nao_controladores.append(balanco_texto[texto])
        print(participacao_dos_nao_controladores)
            
        raspagem = {
            'Ano': anos_nao_repetidos,
            'Ativo Total - (R$)': ativo_total,
            'Ativo Circulante - (R$)': ativo_circulante,
            'Aplicações Financeiras - (R$)': aplicacoes_financeiras,
            'Caixa e Equivalentes de Caixa - (R$)': caixa_e_equivalentes_de_caixa,
            'Contas a Receber - (R$)': contas_a_receber,
            'Estoque - (R$)': estoque, 
            'Ativo Não Circulante - (R$)': ativo_nao_circulante,
            'Ativo Realizável a Longo Prazo - (R$)':ativo_realizavel_a_longo_prazo,
            'Investimentos - (R$)': investimentos,
            'Intangível - (R$)': intangivel, 
            'Passivo Total - (R$)': passivo_total,
            'Passivo Circulante - (R$)': passivo_circulante,
            'Passivo Não Circulante - (R$)':passivo_nao_circulante,
            'Patrimônio Líquido Consolidado - (R$)': patrimonio_liquido_consolidado,
            'Capital Social Realizado - (R$)':capital_social_realizado,
            'Reserva Capital - (R$)': reserva_capital,
            'Reserva Lucros - (R$)': reserva_lucros,
            'Participação dos Não Controladores': participacao_dos_nao_controladores,
        }


        # NOME DA EMPRESA PARA PÔR EM CADA LINHA
        company_description = browser.find_element(By.XPATH, '//div[@class="company-description w-100 w-md-70 ml-md-5"]')
        nome_cnpj = company_description.text.split('\n')
        nome_empresa = nome_cnpj[0]
        nome_empresa = nome_empresa.replace('/', '.')
        cnpj_empresa = nome_cnpj[1]
        
        # ADICIONANDO NOME E CNPJ AO DATAFRAME
        
        raspagem_df = pd.DataFrame(raspagem)

        numero_linhas = raspagem_df.shape[0]
        # numero_linhas

        empresa = []
        for n in range(numero_linhas):
            empresa.append(nome_empresa)
        # empresa
        raspagem_df.insert(0, 'empresa', empresa)

        cnpj = []
        for n in range(numero_linhas):
            cnpj.append(cnpj_empresa)
        # cnpj

        raspagem_df.insert(1, 'cnpj', cnpj)
        # raspagem_df

        display(raspagem_df)
        raspagem_df.to_excel(writer, sheet_name=nome_empresa[:18], header=True, index=False)
        

206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['2.615,45 M', '2.542,23 M', '2.540,60 M', '2.401,12 M', '2.209,43 M']
['484,28 M', '489,29 M', '568,93 M', '417,04 M', '399,74 M']
['0,00', '0,00', '0,00', '0,00', '0,00']
['218,74 M', '127,31 M', '229,09 M', '88,30 M', '73,17 M']
['186,22 M', '254,28 M', '242,58 M', '229,65 M', '228,96 M']
['14,63 M', '11,63 M', '10,01 M', '10,93 M', '5,44 M']
['2.131,17 M', '2.052,94 M', '1.971,67 M', '1.984,08 M', '1.809,69 M']
['600,96 M', '556,66 M', '489,61 M', '489,22 M', '310,81 M']
['4,13 M', '4,37 M', '9,40 M', '8,95 M', '6,63 M']
['537,20 M', '517,12 M', '508,15 M', '519,78 M', '525,02 M']
['988,88 M', '974,79 M', '964,52 M', '966,13 M', '967,24 M']
['2.615,45 M', '2.542,23 M', '2.540,60 M', '2.401,12 M', '2.209,43 M']
['678,39 M', '589,53 M', '476,48 M', '406,23 M', '384,74 M']
['955,15 M', '743,27 M', '840,44 M', '674,89 

,empresa,cnpj,Ano,Ativo Total - (R$),Ativo Circulante - (R$),Aplicações Financeiras - (R$),Caixa e Equivalentes de Caixa - (R$),Contas a Receber - (R$),Estoque - (R$),Ativo Não Circulante - (R$),...,Investimentos - (R$),Intangível - (R$),Passivo Total - (R$),Passivo Circulante - (R$),Passivo Não Circulante - (R$),Patrimônio Líquido Consolidado - (R$),Capital Social Realizado - (R$),Reserva Capital - (R$),Reserva Lucros - (R$),Participação dos Não Controladores
0,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2022,"2.615,45 M","484,28 M","0,00","218,74 M","186,22 M","14,63 M","2.131,17 M",...,"4,13 M","988,88 M","2.615,45 M","678,39 M","955,15 M","981,92 M","612,41 M","614,23 M","0,00","32,22 M"
1,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2021,"2.542,23 M","489,29 M","0,00","127,31 M","254,28 M","11,63 M","2.052,94 M",...,"4,37 M","974,79 M","2.542,23 M","589,53 M","743,27 M","1.209,43 M","612,41 M","613,05 M","0,00","34,14 M"
2,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2020,"2.540,60 M","568,93 M","0,00","229,09 M","242,58 M","10,01 M","1.971,67 M",...,"9,40 M","964,52 M","2.540,60 M","476,48 M","840,44 M","1.223,68 M","612,41 M","622,62 M","0,00","33,15 M"
3,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2019,"2.401,12 M","417,04 M","0,00","88,30 M","229,65 M","10,93 M","1.984,08 M",...,"8,95 M","966,13 M","2.401,12 M","406,23 M","674,89 M","1.320,00 M","612,41 M","620,34 M","49,84 M","37,81 M"
4,ALLIANÇA SAÚDE E PARTICIPAÇÕES S.A.,42.771.949/0001-35,2018,"2.209,43 M","399,74 M","0,00","73,17 M","228,96 M","5,44 M","1.809,69 M",...,"6,63 M","967,24 M","2.209,43 M","384,74 M","524,98 M","1.299,71 M","612,41 M","622,24 M","22,15 M","42,77 M"


206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['26.844,19 M', '23.249,90 M', '13.343,50 M', '8.839,18 M', '7.223,59 M']
['7.722,01 M', '6.867,53 M', '4.210,87 M', '2.273,11 M', '2.293,63 M']
['1.793,22 M', '2.471,03 M', '760,82 M', '309,83 M', '536,39 M']
['1.284,99 M', '1.143,03 M', '753,61 M', '567,81 M', '544,77 M']
['3.303,24 M', '2.305,32 M', '1.743,23 M', '983,79 M', '780,31 M']
['476,03 M', '367,12 M', '357,80 M', '104,64 M', '85,09 M']
['19.122,19 M', '16.382,37 M', '9.132,63 M', '6.566,07 M', '4.929,96 M']
['1.879,08 M', '1.406,33 M', '665,07 M', '367,17 M', '312,56 M']
['3,86 M', '1,03 M', '4,33 M', '5,57 M', '3,98 M']
['6.473,23 M', '5.743,31 M', '3.993,89 M', '2.171,14 M', '1.009,05 M']
['10.766,02 M', '9.231,70 M', '4.469,33 M', '4.022,20 M', '3.604,37 M']
['26.844,19 M', '23.249,90 M', '13.343,50 M', '8.839,18 M', '7.223,59 M']
['6.165,92 M', '6.949,

,empresa,cnpj,Ano,Ativo Total - (R$),Ativo Circulante - (R$),Aplicações Financeiras - (R$),Caixa e Equivalentes de Caixa - (R$),Contas a Receber - (R$),Estoque - (R$),Ativo Não Circulante - (R$),...,Investimentos - (R$),Intangível - (R$),Passivo Total - (R$),Passivo Circulante - (R$),Passivo Não Circulante - (R$),Patrimônio Líquido Consolidado - (R$),Capital Social Realizado - (R$),Reserva Capital - (R$),Reserva Lucros - (R$),Participação dos Não Controladores
0,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2022,"26.844,19 M","7.722,01 M","1.793,22 M","1.284,99 M","3.303,24 M","476,03 M","19.122,19 M",...,"3,86 M","10.766,02 M","26.844,19 M","6.165,92 M","13.588,88 M","7.089,40 M","16.302,24 M","899,14 M","0,00","10,78 M"
1,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2021,"23.249,90 M","6.867,53 M","2.471,03 M","1.143,03 M","2.305,32 M","367,12 M","16.382,37 M",...,"1,03 M","9.231,70 M","23.249,90 M","6.949,67 M","8.751,72 M","7.548,52 M","16.302,24 M","913,71 M","165,20 M","6,23 M"
2,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2020,"13.343,50 M","4.210,87 M","760,82 M","753,61 M","1.743,23 M","357,80 M","9.132,63 M",...,"4,33 M","4.469,33 M","13.343,50 M","3.013,11 M","6.793,86 M","3.536,53 M","12.326,71 M","431,49 M","330,25 M","0,30 M"
3,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2019,"8.839,18 M","2.273,11 M","309,83 M","567,81 M","983,79 M","104,64 M","6.566,07 M",...,"5,57 M","4.022,20 M","8.839,18 M","1.560,81 M","4.014,99 M","3.263,38 M","2.326,42 M","430,35 M","618,25 M","-8,85 M"
4,DIAGNOSTICOS DA AMERICA S.A.,61.486.650/0001-83,2018,"7.223,59 M","2.293,63 M","536,39 M","544,77 M","780,31 M","85,09 M","4.929,96 M",...,"3,98 M","3.604,37 M","7.223,59 M","1.409,25 M","2.492,93 M","3.321,42 M","2.235,37 M","430,35 M","670,92 M","-15,22 M"


206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['7.791,77 M', '6.073,77 M', '5.523,81 M', '5.096,65 M', '3.910,45 M']
['2.391,40 M', '1.742,47 M', '1.880,20 M', '1.485,20 M', '1.434,21 M']
['1.292,23 M', '763,37 M', '1.013,62 M', '795,30 M', '753,65 M']
['17,26 M', '33,72 M', '28,18 M', '8,97 M', '55,23 M']
['856,85 M', '793,85 M', '704,47 M', '570,09 M', '505,42 M']
['96,58 M', '72,61 M', '63,09 M', '31,87 M', '26,56 M']
['5.400,37 M', '4.331,30 M', '3.643,61 M', '3.611,45 M', '2.476,24 M']
['248,19 M', '136,42 M', '139,92 M', '128,45 M', '52,06 M']
['44,08 M', '43,08 M', '34,37 M', '16,83 M', '12,28 M']
['1.948,17 M', '1.565,18 M', '708,77 M', '750,69 M', '782,68 M']
['3.159,93 M', '2.586,61 M', '2.760,55 M', '2.715,49 M', '1.629,23 M']
['7.791,77 M', '6.073,77 M', '5.523,81 M', '5.096,65 M', '3.910,45 M']
['1.234,14 M', '1.206,87 M', '1.109,77 M', '747,04 M', '6

...

,empresa,cnpj,Ano,Ativo Total - (R$),Ativo Circulante - (R$),Aplicações Financeiras - (R$),Caixa e Equivalentes de Caixa - (R$),Contas a Receber - (R$),Estoque - (R$),Ativo Não Circulante - (R$),...,Investimentos - (R$),Intangível - (R$),Passivo Total - (R$),Passivo Circulante - (R$),Passivo Não Circulante - (R$),Patrimônio Líquido Consolidado - (R$),Capital Social Realizado - (R$),Reserva Capital - (R$),Reserva Lucros - (R$),Participação dos Não Controladores
0,ONCOCLINICAS DO BRASIL SERVICOS MEDICOS S.A.,12.104.241/0004-02,2022,"8.219,78 M","2.826,83 M","829,59 M","263,21 M","1.379,61 M","163,94 M","5.392,95 M",...,"22,22 M","3.914,46 M","8.219,78 M","1.883,28 M","3.704,04 M","2.632,46 M","2.145,25 M","805,28 M","-79,40 M","514,67 M"
1,ONCOCLINICAS DO BRASIL SERVICOS MEDICOS S.A.,12.104.241/0004-02,2021,"6.032,41 M","2.871,23 M","1.477,91 M","536,84 M","666,15 M","69,41 M","3.161,18 M",...,"1,39 M","2.238,41 M","6.032,41 M","1.190,72 M","2.536,75 M","2.304,94 M","2.102,67 M","588,01 M","-20,62 M","450,93 M"
2,ONCOCLINICAS DO BRASIL SERVICOS MEDICOS S.A.,12.104.241/0004-02,2020,"3.393,46 M","1.427,35 M","46,21 M","740,89 M","496,55 M","37,07 M","1.966,11 M",...,"0,00","1.198,23 M","3.393,46 M","1.021,30 M","1.691,00 M","681,17 M","416,07 M","466,75 M","0,00","279,40 M"


206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['4.512,03 M', '4.403,58 M', '4.406,98 M', '3.380,06 M', '3.549,31 M']
['1.197,62 M', '1.249,37 M', '1.633,82 M', '925,95 M', '981,67 M']
['333,14 M', '303,08 M', '320,31 M', '290,76 M', '352,09 M']
['377,13 M', '449,46 M', '418,77 M', '201,05 M', '137,45 M']
['232,04 M', '221,88 M', '711,49 M', '207,24 M', '192,36 M']
['0,00', '0,00', '0,00', '0,00', '-']
['3.314,41 M', '3.154,21 M', '2.773,16 M', '2.454,12 M', '2.567,65 M']
['327,38 M', '231,96 M', '255,40 M', '179,39 M', '107,51 M']
['126,43 M', '131,33 M', '0,26 M', '0,26 M', '0,26 M']
['85,94 M', '55,11 M', '43,91 M', '133,96 M', '54,51 M']
['2.774,65 M', '2.735,82 M', '2.473,59 M', '2.140,50 M', '2.405,37 M']
['4.512,03 M', '4.403,58 M', '4.406,98 M', '3.380,06 M', '3.549,31 M']
['674,38 M', '2.618,13 M', '1.142,79 M', '471,14 M', '1.047,89 M']
['2.481,06 M', '51

,empresa,cnpj,Ano,Ativo Total - (R$),Ativo Circulante - (R$),Aplicações Financeiras - (R$),Caixa e Equivalentes de Caixa - (R$),Contas a Receber - (R$),Estoque - (R$),Ativo Não Circulante - (R$),...,Investimentos - (R$),Intangível - (R$),Passivo Total - (R$),Passivo Circulante - (R$),Passivo Não Circulante - (R$),Patrimônio Líquido Consolidado - (R$),Capital Social Realizado - (R$),Reserva Capital - (R$),Reserva Lucros - (R$),Participação dos Não Controladores
0,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2022,"4.512,03 M","1.197,62 M","333,14 M","377,13 M","232,04 M","0,00","3.314,41 M",...,"126,43 M","2.774,65 M","4.512,03 M","674,38 M","2.481,06 M","1.356,59 M","875,58 M","12,23 M","458,64 M","10,14 M"
1,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2021,"4.403,58 M","1.249,37 M","303,08 M","449,46 M","221,88 M","0,00","3.154,21 M",...,"131,33 M","2.735,82 M","4.403,58 M","2.618,13 M","510,41 M","1.275,05 M","875,58 M","4,69 M","387,87 M","8,63 M"
2,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2020,"4.406,98 M","1.633,82 M","320,31 M","418,77 M","711,49 M","0,00","2.773,16 M",...,"0,26 M","2.473,59 M","4.406,98 M","1.142,79 M","1.578,82 M","1.685,38 M","875,58 M","122,25 M","685,50 M","2,05 M"
3,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2019,"3.380,06 M","925,95 M","290,76 M","201,05 M","207,24 M","0,00","2.454,12 M",...,"0,26 M","2.140,50 M","3.380,06 M","471,14 M","1.482,09 M","1.426,84 M","841,91 M","14,40 M","425,49 M",-
4,QUALICORP CONSULTORIA E CORRETORA DE SEGUROS S.A.,11.992.680/0001-93,2018,"3.549,31 M","981,67 M","352,09 M","137,45 M","192,36 M",-,"2.567,65 M",...,"0,26 M","2.405,37 M","3.549,31 M","1.047,89 M","146,94 M","2.354,49 M","1.809,07 M","1,12 M","399,26 M",-


206
103
['2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018', '2022', '2021', '2020', '2019', '2018']
['2022', '2021', '2020', '2019', '2018']
['92.490,68 M', '52.480,62 M', '45.657,04 M', '26.599,67 M', '19.028,90 M']
['41.677,69 M', '21.889,58 M', '22.431,90 M', '9.729,22 M', '8.054,76 M']
['29.236,65 M', '12.429,55 M', '15.580,37 M', '4.701,42 M', '3.864,70 M']
['1.109,80 M', '124,62 M', '146,88 M', '68,68 M', '192,01 M']
['6.808,57 M', '7.564,77 M', '5.163,87 M', '4.117,92 M', '3.259,35 M']
['808,69 M', '735,25 M', '607,95 M', '335,48 M', '284,43 M']
['50.812,99 M', '30.591,04 M', '23.225,14 M', '16.870,44 M', '10.974,15 M']
['14.457,71 M', '5.402,66 M', '3.959,05 M', '1.468,60 M', '1.022,18 M']
['2.553,40 M', '2.325,84 M', '1.089,23 M', '1.226,88 M', '144,53 M']
['14.970,85 M', '12.231,50 M', '10.366,97 M', '8.809,50 M', '4.994,83 M']
['18.831,04 M', '10.631,03 M', '7.809,90 M', '5.365,47 M', '4.812,60 M']
['92.490,68 M', '52.480,62 M', '45.657,04 M', '2

,empresa,cnpj,Ano,Ativo Total - (R$),Ativo Circulante - (R$),Aplicações Financeiras - (R$),Caixa e Equivalentes de Caixa - (R$),Contas a Receber - (R$),Estoque - (R$),Ativo Não Circulante - (R$),...,Investimentos - (R$),Intangível - (R$),Passivo Total - (R$),Passivo Circulante - (R$),Passivo Não Circulante - (R$),Patrimônio Líquido Consolidado - (R$),Capital Social Realizado - (R$),Reserva Capital - (R$),Reserva Lucros - (R$),Participação dos Não Controladores
0,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2022,"92.490,68 M","41.677,69 M","29.236,65 M","1.109,80 M","6.808,57 M","808,69 M","50.812,99 M",...,"2.553,40 M","18.831,04 M","92.490,68 M","16.743,80 M","52.669,91 M","23.076,97 M","15.458,33 M","4.374,11 M","1.669,51 M","1.264,93 M"
1,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2021,"52.480,62 M","21.889,58 M","12.429,55 M","124,62 M","7.564,77 M","735,25 M","30.591,04 M",...,"2.325,84 M","10.631,03 M","52.480,62 M","5.149,03 M","32.653,65 M","14.677,94 M","7.322,49 M","4.340,45 M","1.600,18 M","1.094,56 M"
2,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2020,"45.657,04 M","22.431,90 M","15.580,37 M","146,88 M","5.163,87 M","607,95 M","23.225,14 M",...,"1.089,23 M","7.809,90 M","45.657,04 M","3.861,05 M","27.793,81 M","14.002,18 M","6.472,95 M","4.421,25 M","2.789,48 M","389,06 M"
3,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2019,"26.599,67 M","9.729,22 M","4.701,42 M","68,68 M","4.117,92 M","335,48 M","16.870,44 M",...,"1.226,88 M","5.365,47 M","26.599,67 M","2.692,84 M","18.464,82 M","5.442,00 M","867,78 M","1.874,87 M","2.467,48 M","231,87 M"
4,REDE D OR SÃO LUIZ S.A.,06.047.087/0001-39,2018,"19.028,90 M","8.054,76 M","3.864,70 M","192,01 M","3.259,35 M","284,43 M","10.974,15 M",...,"144,53 M","4.812,60 M","19.028,90 M","2.181,24 M","12.319,94 M","4.527,73 M","867,78 M","1.874,81 M","1.592,93 M","192,21 M"
